In [1]:
import numpy as np
import matplotlib.pyplot as plt
import IPython
from scipy.io import wavfile
from scipy import signal

# Tone Generation

In [2]:
def gentone(num):
    '''outputs a two-tone wave based on the input n'''
    '''n is a single digit whole number, or the a string of * or #'''
    x = [1] + [0]*2998 # sampling frequency is 8kHz, so 5000 samples will be used to generate a wave 5/8 = 0.625 seconds long
    y1 = [0]*len(x)
    y2 = [0]*len(x)
    wave = [0]*len(x)
    
    if num == 1:
        f1 = 697
        f2 = 1209
    elif num == 2:
        f1 = 697
        f2 = 1336
    elif num == 3:
        f1 = 697
        f2 = 1477
    elif num == 4:
        f1 = 770
        f2 = 1209
    elif num == 5:
        f1 = 770
        f2 = 1336
    elif num == 6:
        f1 = 770
        f2 = 1477
    elif num == 7:
        f1 = 852
        f2 = 1209
    elif num == 8:
        f1 = 852
        f2 = 1336
    elif num == 9:
        f1 = 852
        f2 = 1477
    elif num == '*':
        f1 = 941
        f2 = 1209
    elif num == 0:
        f1 = 941
        f2 = 1336
    elif num == '#':
        f1 = 941
        f2 = 1477
    
    fsamp = 8000
    omega1 = 2*np.pi*f1/fsamp
    omega2 = 2*np.pi*f2/fsamp
    
    '''generate two single-tone waves for the input'''
    for n in range(len(x)):
        if n< len(x):
            y1[n] = x[n-1]*np.sin(omega1) + 2*y1[n-1]*np.cos(omega1) - y1[n-2]
            y2[n] = x[n-1]*np.sin(omega2) + 2*y2[n-1]*np.cos(omega2) - y2[n-2]
    
    '''add the two waves and amplify them'''
    for i in range(len(wave)):
        wave[i] = 12500*(y1[i] + y2[i])
    return wave

In [3]:
phone1 = gentone(1)
gen1 = np.asarray(phone1, dtype=np.int16)
wavfile.write('1 twotone.wav',8000,gen1)
IPython.display.Audio("1 twotone.wav")

In [4]:
phone2 = gentone(2)+gentone(2)
gen2 = np.asarray(phone2, dtype=np.int16)
wavfile.write('2 twotone.wav',8000,gen2)
IPython.display.Audio("2 twotone.wav")

In [5]:
phone8 = gentone(8)
gen8 = np.asarray(phone8, dtype=np.int16)
wavfile.write('8 twotone.wav',8000,gen8)
IPython.display.Audio("8 twotone.wav")

In [6]:
silence = [0]*1500

KPUPhone = gentone(6)+silence+gentone(0)+silence+gentone(4)+silence + gentone(5)+silence+gentone(9)+silence+gentone(9)+silence + gentone(2)+silence+gentone(0)+silence+gentone(0)+silence+gentone(0)
genKPU = np.asarray(KPUPhone, dtype=np.int16)
wavfile.write('KPU twotone.wav',8000,genKPU)
IPython.display.Audio("KPU twotone.wav")

# Reconstruction

In [7]:
def tone2num(wave):
    '''function takes a single waveform consisting of 2 frequencies and returns a number or string according to the DTMF table'''
    Amin = 2000 # the minimum amplitude of a frequency for it to be registered as a component of the wave
    num = 0 # initializing the output variable (can be 0-9, #, or *)
    N = len(wave)
    fsamp = 8000
    freqs = np.linspace(0, fsamp, N)
    ffttwotone = np.abs(np.fft.fft(wave))/N #take fourier transform
    
    '''check amplitudes of each relevant frequency in the DTMF table by checking a bandwidth'''
    frequency = [697, 770, 852, 941, 1209, 1336, 1477]
    check = [0]*len(frequency) # This array will be used to check the presence of each frequency by
    for f in range(7):
        b, a = signal.butter(4, [(frequency[f]-5)/4000,(frequency[f]+5)/4000], btype = 'bandpass')
        filt = signal.filtfilt(b, a, wave)
        ffttwotone = np.abs(np.fft.fft(wave))/N #take fourier transform
        if max(filt) > Amin:
            check[f] = 1 # set a marker in the "check" array to denote if a frequency was detected
    
    '''the "check" array is a list of 7 numbers, each corresponsing to a frequency that makes up the dual tones. The first 4 items correspoond to the frequencies 697, 770, 852, 941, the vertical coordinates of the DTMF table'''
    '''the last 3 items correspond to the higher frequencies that make up the horizontal coordinates on the DTMF table'''
    if   check == [1, 0, 0, 0, 1, 0, 0]:
        num = 1
        print(f'The keypad entry for this tone was {num}.')
    elif check == [1, 0, 0, 0, 0, 1, 0]:
        num = 2
        print(f'The keypad entry for this tone was {num}.')
    elif check == [1, 0, 0, 0, 0, 0, 1]:
        num = 3
        print(f'The keypad entry for this tone was {num}.')
    elif check == [0, 1, 0, 0, 1, 0, 0]:
        num = 4
        print(f'The keypad entry for this tone was {num}.')
    elif check == [0, 1, 0, 0, 0, 1, 0]:
        num = 5
        print(f'The keypad entry for this tone was {num}.')
    elif check == [0, 1, 0, 0, 0, 0, 1]:
        num = 6
        print(f'The keypad entry for this tone was {num}.')
    elif check == [0, 0, 1, 0, 1, 0, 0]:
        num = 7
        print(f'The keypad entry for this tone was {num}.')
    elif check == [0, 0, 1, 0, 0, 1, 0]:
        num = 8
        print(f'The keypad entry for this tone was {num}.')
    elif check == [0, 0, 1, 0, 0, 0, 1]:
        num = 9
        print(f'The keypad entry for this tone was {num}.')
    elif check == [0, 0, 0, 1, 1, 0, 0]:
        num = '*'
        print(f'The keypad entry for this tone was {num}.')
    elif check == [0, 0, 0, 1, 0, 1, 0]:
        num = 0
        print(f'The keypad entry for this tone was {num}.')
    elif check == [0, 0, 0, 1, 0, 0, 1]:
        num = '#'
    else:
        print('Wave is incompatible with function')
        
    return
    

In [8]:
tone2num(phone8)

The keypad entry for this tone was 8.


In [9]:
tone2num(gentone('*'))

The keypad entry for this tone was *.
